In [1]:
import re
import requests
from html.parser import HTMLParser

In [11]:
class Parser8035(HTMLParser):
    url = 'https://www.tel.co.jp/news/index.html'
    url_domain = 'https://www.tel.co.jp'
    def __init__(self):
        HTMLParser.__init__(self)

        self.c_news = False
        self.c_news__col = False
        self.time = False
        self.summary = False

        self.result = list()
        self.element = None
        self.link = None

    def handle_starttag(self, tag, attrs):
        dict_attrs = dict(attrs)
        if tag == 'div':
            if 'class' in dict_attrs:
                if dict_attrs['class'] == 'c-news':
                    self.c_news = True
                    self.element = list()
                    return
                if dict_attrs['class'] == 'c-news__col':
                    self.c_news__col = True
                    return
                return
            return

        if tag == 'a' and self.c_news is True:
            if 'class' in dict_attrs:
                if dict_attrs['class'] == 'u-hover':
                    self.link = '%s%s' % (self.url_domain, dict_attrs['href'])
                    return
                return
            return

        if tag == 'time' and self.c_news__col is True:
            self.time = True
            return

        if tag == 'p' and self.c_news__col is True:
            if 'class' in dict_attrs:
                if dict_attrs['class'] == 'c-news__summary u-fileicon':
                    self.summary = True
                    return
                return
            return
            

    def handle_data(self, data):
        content = data.strip()

        if self.time is True and self.c_news__col is True:
            self.element.append(content)
            self.time = False
            return

        if self.summary is True and self.c_news__col is True:
            self.element.append(content)
            self.element.append(self.link)
            self.summary = False
            self.link = None
            return


    def handle_endtag(self, tag):
        if tag == 'div':
            if self.c_news__col is True:
                self.c_news__col = False
                return

            if self.c_news is True:
                if len(self.element) > 0:
                    self.result.append(self.element)
                self.c_news = False
                return

In [12]:
parser = Parser8035()
url = parser.url
response = requests.get(url)

In [13]:
parser.feed(response.text)
parser.close()

for line in parser.result:
    print(line)

['2024.05.17', 'Institutional Investor誌の「All-Japan Executive Team Ranking」(ベストIR企業ランキング) の「Most Honored Company」に9年連続で選定', 'https://www.tel.co.jp/news/ir/2024/20240517_001.html']
['2024.05.17', 'ITF World2024', 'https://www.tel.co.jp/news/event/2024/20240517_001.html']
['2024.05.10', '自己株式の取得に係る事項の決定に関するお知らせ', 'https://www.tel.co.jp/news/ir/2024/r1apfs00000000mh-att/20240510_003.pdf']
['2024.05.10', '株式報酬として新株予約権を発行する件', 'https://www.tel.co.jp/news/ir/2024/r1apfs00000000kn-att/20240510_002.pdf']
['2024.05.10', '剰余金の配当（2024年3月期期末配当）に関するお知らせ', 'https://www.tel.co.jp/news/ir/2024/r1apfs00000000g0-att/20240510_001.pdf']
['2024.05.10', '2024年3月期 決算発表', 'https://www.tel.co.jp/ir/library/report/index.html']
['2024.04.23', '「SX銘柄 2024」に選定', 'https://www.tel.co.jp/news/sustainability/2024/20240423_001.html']
['2024.04.03', '台湾東部を震源として発生した地震について', 'https://www.tel.co.jp/news/disaster/2024/20240403_001.html']
['2024.04.02', '「健康経営優良法人2024」の上位500法人に認定', 'https://www.tel.co.jp/news/topics/2024/2024